# Find deviations from Lichess masters DB

In [13]:
import os
import time
import requests
from dotenv import load_dotenv

load_dotenv()

token = os.environ.get("LICHESS_TOKEN")

In [14]:
import requests
import chess
import chess.pgn
import time
import sys
import berserk
import io
from IPython.display import SVG
from tqdm import tqdm

session = berserk.TokenSession(token)
client = berserk.Client(session)

In [15]:
# Check FEN against Lichess masters DB
def check_fen_against_mastersdb(fen):

  FENdict = {}
  cachedfen = FENdict.get(fen)

  if cachedfen:
    r = cachedfen

  else:
    while True:
      payload = {'fen': fen, 'topGames': 0, 'moves': 30}
      r = requests.get(f'https://explorer.lichess.ovh/master', params = payload)
      if r.status_code == 200:
        time.sleep(0.2)
        r = r.json()
        break
      if r.status_code == 429:
        print("Waiting 5 seconds")
        time.sleep(5)
        continue
    # FENdict[fen] = r
    matches = r['white'] + r['black'] + r['draws']
  return matches

fen = 'rn1qkb1r/pp3ppp/2p1pn2/3p1b2/2PP4/1Q3NP1/PP2PPBP/RNB1K2R b KQkq -'
check_fen_against_mastersdb(fen)

20

In [17]:
# Identify when a Lichess game was last in "theory" (aka matched at least one masters DB game) 
def find_last_ply_found_in_mastersdb(id, start_ply = 0):

  pgn = client.games.export(id, as_pgn=True)
  game_num = 0
  
  while True:
    with io.StringIO(pgn) as f:
      game = chess.pgn.read_game(f)
    if not game or game_num >= 1:
      break
    game_num += 1
    board = game.board()
    masters_matches = 99999
    while masters_matches > 0:
      for n, move in enumerate(game.mainline_moves()):
        if n < start_ply:
            board.push(move)
        else:
            masters_matches = check_fen_against_mastersdb(board.epd())
            if masters_matches == 0: # stop when no matches are found
                break
            else:
                board.push(move)  
  return n-1

find_last_ply_found_in_mastersdb('5fULpU0u')

12

In [18]:
# Check a ply in a game against the masters DB and return the number of matches
def check_ply_against_mastersdb(id, ply):
    pgn = client.games.export(id, as_pgn=True)
    game_num = 0
    while True:
        with io.StringIO(pgn) as f:
            game = chess.pgn.read_game(f)
        if not game or game_num >= 1:
            break
        game_num += 1
        board = game.board()
        
        for n, move in enumerate(game.mainline_moves()):
            if n < ply:
                board.push(move)  
            else:
                matches = check_fen_against_mastersdb(board.epd())
                break        
    return matches

In [19]:
# Find game with the latest ply matched in the masters DB
def find_latest_deviation_from_mastersdb(ids):
    
    latest_deviation_ply = 9999
    latest_deviation_id = ''
    
    for i in range(0, len(ids)):
        print('Checking game', i+1, '/', len(ids), ':', ids[i])

        if i == 0:
            latest_deviation_ply = find_last_ply_found_in_mastersdb(ids[i], 1)
            latest_deviation_id = ids[i]
            print('Current latest ply matched in masters DB:', latest_deviation_ply, 'in', latest_deviation_id)

        if i > 0:
            new_matches = check_ply_against_mastersdb(ids[i], latest_deviation_ply)
            if new_matches == 0:
                continue
            if new_matches > 0:
                latest_deviation_ply = find_last_ply_found_in_mastersdb(ids[i], latest_deviation_ply)
                latest_deviation_id = ids[i]
                print('New latest ply matched in masters DB:', latest_deviation_ply, 'in', latest_deviation_id)

        if i > 0 and i % 100 == 0:
            print('Current latest ply matched in masters DB:', latest_deviation_ply, 'in', latest_deviation_id)

    return [latest_deviation_id, latest_deviation_ply]


find_latest_deviation_from_mastersdb(['0oht8pGP', 'JSyE55lM', '42UIwlRN', 'YsMWRKcn', '2QnQM1ns'])

# More games to test...
# 'rpT0BJkr', 'elDXjTwi', 'HqYHBLxl', 'LmSs5zO5', 'zssiRcV2',
# 'zssiRcV2', 'QqY5HUCU', 'BSB32D9z', '5lFmsjoC', 'SG4cXHBZ',
# '45KDSsll'

Checking game 1 / 5 : 0oht8pGP
Current latest ply matched in masters DB: 16 in 0oht8pGP
Checking game 2 / 5 : JSyE55lM
Checking game 3 / 5 : 42UIwlRN
New latest ply matched in masters DB: 17 in 42UIwlRN
Checking game 4 / 5 : YsMWRKcn
New latest ply matched in masters DB: 17 in YsMWRKcn
Checking game 5 / 5 : 2QnQM1ns
New latest ply matched in masters DB: 20 in 2QnQM1ns


['2QnQM1ns', 20]